In [84]:
from functools import partial

import astroplan as ap
from astropy.time import Time
import astropy.units as u
import pandas as pd

from astropaul.database import database_connection, html_path
import astropaul.targetlistcreator as tlc
import astropaul.html as html

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
session = tlc.ObservingSession(ap.Observer.at_site("Gemini South"))
t = Time("2026-01-01", format="iso")
while t < Time("2026-12-31", format="iso"):
    session.add_full_day(t.iso[:10])
    t += 7 * u.day

name = f"2026 Tokovinin Targets"

html_dir = html_path() / name
html.clear_directory(html_dir)

creator = tlc.TargetListCreator(name=name)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])),
    # partial(tlc.filter_targets, criteria=lambda df: df["Target Source"].str.startswith("Kostov")),
    tlc.add_lists,
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["List Table 1 Targets"]),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["List Table 2 Targets"]),
    tlc.ancillary_data_from_tess,
    partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] < 12),
    partial(tlc.add_observability, observing_session=session, time_resolution=2 * u.hour),
    partial(tlc.filter_targets, criteria=lambda df: df["Observable Any Night"]),
    partial(tlc.add_database_table, table_name="DSSI Observations"),
    partial(tlc.filter_targets, criteria=lambda df: df["Num DSSI Observations"] == 0),
    partial(tlc.add_database_table, table_name="Ephemerides"),
    tlc.tabulate_ephemerides,
]

with database_connection() as conn:
    tl = creator.calculate(connection=conn, verbose=False)

tl.target_list.sort_values(["Num Ephemerides", "Target Name"], ascending=[False, True], inplace=True)

html.render_observing_pages(tl, None, {}, html_dir)

print(len(tl.target_list))
print(tl.summarize())

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator\observability.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.column_groups[column_prefix.strip()] = (main_columns, nightly_columns)
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator\observability.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.column_groups[column_prefix.strip()] = (main_columns, nightly_columns)
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator\observability.py:95: PerformanceWarning: DataF

23
Name: 2026 Tokovinin Targets
Criteria:
  All targets loaded (732 targets remain)
  lambda df: df["Target Type"].isin(["QuadEB", "SextEB"]) (322 targets remain)
  Inverse of: lambda df: df["List Table 1 Targets"] (288 targets remain)
  Inverse of: lambda df: df["List Table 2 Targets"] (265 targets remain)
  lambda df: df["Vmag"] < 12 (53 targets remain)
  Observability calculated at Gemini South in 2.0 h intervals from 2025-12-31 to 2026-12-24
    AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
  lambda df: df["Observable Any Night"] (35 targets remain)
  lambda df: df["Num DSSI Observations"] == 0 (23 targets remain)
  
23 targets:
    21 QuadEB
     2 SextEB
Column Count (primary, secondary):
    Target: (3, 4)
    List: (0, 18)
    TESS Data: (4, 0)
    Observable: (4, 156)
    Count: (2, 0)
Associated tables:
     968 rows,  2 columns: List Memberships
     733 rows, 126 columns: TESS
      52 rows,  2 columns: Lunar Phases
      19 rows,  9 columns: DS

In [ ]:
tk = pd.DataFrame()
tk["Target Name"] = tl.target_list["Target Name"].str.replace(" ", "")
tk["RA2000"] = tl.target_list["RA HMS"]
tk["Dec2000"] = tl.target_list["Dec DMS"]
tk["PM RA"] = tl.target_list["PMra"]
tk["PM Dec"] = tl.target_list["PMdec"]
tk["Magnitude"] = tl.target_list["Vmag"]
tk["Priority"] = pd.cut(tl.target_list["Num Ephemerides"], bins=[0, 2, 3, 6], labels=["*", "**", "***"])
tk.to_csv("2026 QuadEB SOAR Targets.csv", index=False)
tk


,Target Name,RA2000,Dec2000,PM RA,PM Dec,Magnitude,Priority
5,TIC140328928,07:43:17.21,-23:38:22.41,-1.507610,3.820200,11.1570,***
0,TIC311838200,12:07:26.87,-70:29:21.7,12.100000,1.700000,10.3900,***
1,TIC314802266,12:19:43.73,-71:29:55.9,-9.016670,-4.223840,11.7560,***
8,TIC320233974,11:38:41.42,-62:49:07.36,-5.651290,0.640360,10.7340,***
9,TIC321474625,11:41:40.97,-62:36:26.69,-5.349200,1.703800,11.2750,***
12,TIC444816203,17:51:36.42,-32:02:01.34,0.705236,-1.537410,9.3790,***
13,TIC459333241,10:49:14.74,-59:46:52.49,-6.891980,3.197060,11.0020,***
3,TIC125583594,07:03:23.44,-06:52:11.77,-0.456588,0.003471,10.4030,**
4,TIC139621206,10:06:52.05,-51:04:39.8,-21.563500,7.355940,7.0900,**
6,TIC153406662,06:12:54.95,+09:02:08.86,-4.008320,-3.020660,7.9600,**
